# ***MODOAP - Calcul de similarité entre images***

Ce notebook permet de calculer la similarité entre une image donnée et les images d'un corpus présent sur un Google Drive.
Il dispose de trois fonctionnalités :
1. Vectorisation d'un corpus / création d'un modèle à requêter
2. Requête des n plus proches images sur un modèle importé, à partir d'une image donnée en entrée.
3. Repérage et sauvegarde des doublons

Il implémente la bibliothèque turicreate : https://github.com/apple/turicreate

**Le mode GPU est nécessaire (Runtime -> Change Runtime Type -> GPU)**

## 0. Connexion à un compte Google Drive, création de l'architecture et installation des pré-requis

Nécessite de se connecter à son compte Google Drive et d'entrer un code de vérification.

Crée un dossier Outils_Modoap sur le Drive qui servira à stocker les modèles générés et les doublons repérés.

In [ ]:
from google.colab import drive
import os

if not os.path.exists("/content/drive/My Drive"):
  drive.mount('/content/drive')
else : print("Le Drive est déjà monté")

if not os.path.exists("/content/drive/My Drive/Outils_Modoap/Similarite"):
  os.makedirs('/content/drive/My Drive/Outils_Modoap/Similarite/Doublons')
  os.makedirs('/content/drive/My Drive/Outils_Modoap/Similarite/Modeles')

!apt install libnvrtc8.0
!pip install turicreate

import turicreate as tc
from IPython.display import display
from IPython.display import HTML
from PIL import Image
from io import BytesIO
import glob
import random
import base64
import pandas as pd
from zipfile import ZipFile
from shutil import copy
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files
from google.colab import drive
from time import gmtime, strftime
import shutil

def get_thumbnail(path):
  i = Image.open(path)
  i.thumbnail((150, 150), Image.LANCZOS)
  return i
  
def get_thumbnail_from_image(img):
  i = img.copy()
  i.thumbnail((150, 150), Image.LANCZOS)
  return i

def image_base64(im):
  if isinstance(im, str):
    im = get_thumbnail(im)
  with BytesIO() as buffer:
    im.save(buffer, 'jpeg')
    return base64.b64encode(buffer.getvalue()).decode()

def image_formatter(im):
  return f'<img style="display:inline;margin:1px" src="data:image/jpeg;base64,{image_base64(im)}">'
  
def preview_images(reference_data, num_previews=30):
  images = list(map(lambda x:image_formatter(get_thumbnail(x)), reference_data[0:num_previews]['path']))
  display(HTML(''.join(images)))

# Use all GPUs (default)
tc.config.set_num_gpus(-1) 

# Use only 1 GPU
#tc.config.set_num_gpus(1)

# Use CPU
#tc.config.set_num_gpus(0)

## 1. Préparation d'un corpus

Le corpus doit être sous forme d'un dossier contenant directement les images. 

Si les images du corpus sont réparties en classes, elles peuvent être rangées dans des dossiers dont les noms sont identiques aux classes.

Deux possibilités au choix pour importer le corpus :

- 1.1 Utiliser un corpus présent sur le drive
- 1.2 Télécharger un mini corpus exemple

**1.1 Utiliser un corpus présent sur le drive**

Lancer la cellule et entrer le chemin absolu du dossier sur le drive contenant le corpus. La racine du Google Drive connecté est /content/drive/My Drive/

Possibilité de copier/coller le chemin depuis la fenêtre de gauche : Files -> clic droit sur un dossier -> Copy Path

Exemple de chemin:

/content/drive/My Drive/Corpus/corpus_pour_similarite/ 

In [ ]:
corpus = input("Entrer le chemin absolu du dossier contenant le corpus")

**1.2 Télécharger un mini corpus exemple**

Télécharge le corpus sur le drive

In [ ]:
if not os.path.exists("/content/drive/My Drive/Outils_Modoap/Corpus_demonstrations/corpus_similarite/"):
  os.makedirs('/content/drive/My Drive/Outils_Modoap/Corpus_demonstrations/corpus_similarite/')
%cd /content/drive/My Drive/Outils_Modoap/Corpus_demonstrations/corpus_similarite/
!wget https://github.com/MODOAP/corpus_test/releases/download/1/simcorp.zip
!7z x ./simcorp.zip
os.remove("/content/drive/My Drive/Outils_Modoap/Corpus_demonstrations/corpus_similarite/simcorp.zip")
corpus = "/content/drive/My Drive/Outils_Modoap/Corpus_demonstrations/corpus_similarite/"

**1.3 Création et sauvegarde du modèle à partir du corpus**

Enregistre le modèle sur le drive dans /content/drive/My Drive/Outils_Modoap/Similarite/Modeles/

In [ ]:
# Load images from a folder
reference_data = tc.image_analysis.load_images(corpus)
reference_data = reference_data.add_row_number()
# From the path-name, create a label column
reference_data['label'] = reference_data['path'].apply(lambda path: path.split('/')[-2])
reference_data.groupby('label', [tc.aggregate.COUNT]).sort("Count", ascending = False)
# Save the SFrame for future use
reference_data.save('/content/drive/My Drive/Outils_Modoap/Similarite/Modeles/reference_data.sframe')
reference_data.groupby('label', [tc.aggregate.COUNT]).sort("Count", ascending = False)
reference_data.head()

# create an image similarity model using the data
model = tc.image_similarity.create(reference_data)
# saving the model
%cd /content/drive/My Drive/Outils_Modoap/Similarite/Modeles
nom_modele = "ModeleSimilarite_"+str(strftime("%Y-%m-%d %H:%M:%S", gmtime())).replace(" ","_")+".zip"
model.save('./image_similarity.model')
!zip -r ./$nom_modele ./image_similarity.model ./reference_data.sframe
!rm -r ./image_similarity.model
!rm -r ./reference_data.sframe

**1.4 Prévisualisation du corpus** (optionnel)

Prévisualiser un certain nombre d'images du corpus : spécifier le nombre


In [ ]:
nbimage = input("Spécifiez le nombre d'images à prévisualiser : ")
preview_images(reference_data, int(nbimage))

##2. Requêtes d'images similaires sur un modèle


**2.1 Importer un modèle depuis son google Drive :** (Optionnel)

Importer un modèle précédemment sauvegardé sur le Drive, sous forme d'un fichier .zip

Si le modèle vient d'être créé et sauvegardé via l'étape 1, il n'est pas nécessaire de l'importer

Exemple de chemin : /content/drive/My Drive/Outils_Modoap/Similarite/Modeles/monmodele.zip


In [ ]:
fichier = input("Entrer le chemin absolu du fichier modèle .zip")
%cd /content/drive/My Drive/Outils_Modoap/Similarite/Modeles
ds = ZipFile(fichier)
ds.extractall()
print("Modèle téléchargé et extrait")

reference_data = tc.load_sframe('/content/drive/My Drive/Outils_Modoap/Similarite/Modeles/reference_data.sframe')
model = tc.load_model('/content/drive/My Drive/Outils_Modoap/Similarite/Modeles/image_similarity.model')
print("Modèle et Sframe importés")

**2.2 Spécifier une image source**

Une requête sur le modèle s'effectue à partir d'une image source. Lancer la cellule et spécifier l'url d'une image sur le web ou sur le Drive. 

Exemples :

https://www.parismuseescollections.paris.fr/sites/default/files/styles/pm_notice/public/atoms/images/CAR/lpdp_77897-18.jpg

ou :

/content/drive/My Drive/Outils_Modoap/Corpus_demonstrations/corpus_similarite/BDIC_KAG_02679N_A27.jpg



In [ ]:
image_url = input("Spécifiez l'url web de l'image : ")

**2.3 Lancer la requête**

Lancer la cellule et spécifier le nombre d'images les plus similaires à afficher.

Les doublons seront sauvegardés dans /content/drive/My Drive/Outils_Modoap/Similarite/Doublons/

Les n plus proches voisins seront affichés si le corpus a été importé.
Sinon, seuls les noms seront retournés.

In [ ]:
print("url de l'image : ", image_url)
sample_image = tc.Image(image_url)
si_height = sample_image.height 
si_width = sample_image.width
if int(si_height) > 600 :
  sample_image = tc.image_analysis.resize(sample_image, 320, 240)
  display(sample_image)
else : 
  display(sample_image)

userk = input("Spécifier le nombre d'images les plus similaires à afficher : ")

print("-----------------------------------------")

query_results = model.query(sample_image, k=int(userk))
similar_rows = query_results[query_results['query_label'] == 0]['reference_label']
doublons_potentiels = [idi for idi in similar_rows if query_results[query_results['reference_label'] == idi]['distance'][0] < 10]
similar_rows = [i for i in similar_rows if i not in doublons_potentiels]
doublons_data = reference_data.filter_by(doublons_potentiels, 'id')
similar_rows_data = reference_data.filter_by(similar_rows, 'id')
similar_rows_data = similar_rows_data.filter_by(doublons_potentiels, 'id', exclude = True)
if len(doublons_data) >= 1 :
  print("--------------------------------------------------------------")
  print("Des doublons ont été repérés dans le corpus : ")
  print("--------------------------------------------------------------")
  preview_images(doublons_data)

  for doublon in doublons_potentiels :
    chem1 = doublons_data[doublons_data['id'] == doublon]['path'][0]
    nomf = "/content/drive/My Drive/Outils_Modoap/Similarite/Doublons/"+str(chem1.split("/")[-1])
    shutil.copyfile(chem1, nomf)
  print("Les doublons ont été sauvés dans /content/drive/My Drive/Outils_Modoap/Similarite/Doublons ")

print("--------------------------------------------------------------")
print("Les {} plus proches voisins sont : ".format(userk))
print("--------------------------------------------------------------")
if os.path.isfile(similar_rows_data[0]["path"]) :
  for idi in similar_rows :
    chem1 = similar_rows_data[similar_rows_data['id'] == idi]['path'][0]   
    piil = Image.open(chem1)
    dist = query_results[query_results['reference_label'] == idi]['distance'][0]
    print("index : ",idi)
    print("chemin : ", chem1)
    print("distance : ", dist)
    display(piil)
else :
  for row in similar_rows_data :
    print(row["path"])
  print("(Le corpus n'a pas été importé, impossible d'afficher les images) ")